Installing requried packages

In [55]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [56]:
pip install sklearn_crfsuite

Note: you may need to restart the kernel to use updated packages.


Importing all of the needed packages

In [57]:
import nltk
from sklearn_crfsuite import CRF
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers
import re
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
import scipy
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

In [58]:
#Downloading the requried data
nltk.download('treebank')
nltk.download('omw-1.4')
nltk.download('universal_tagset')
nltk.download('wordnet')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\millermicd\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\millermicd\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\millermicd\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\millermicd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [59]:
#loading the data
 
tagged_sentence = nltk.corpus.treebank.tagged_sents(tagset='universal')

In [60]:
# checking what all the labels are for this dataset
lab = set([label for pair in tagged_sentence for word, label in pair])       
lab

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

In [61]:
# spliting into training and testing data
train, test = train_test_split(tagged_sentence, test_size = .25, random_state = 42 )

we can try and use int() to make values of 0,1 for False and true values. if int(True) = 1 ,int(False) = 0
added has_apostrophe to the features to try and make conjunctions better. All of the other features I found were good to use from research into this topic

In [62]:
def get_features(sentence, index):
    return {
        'is_first_capital':int(sentence[index][0].isupper()),
        'is_first_word': int(index==0),
        'is_last_word':int(index==len(sentence)-1),
        'is_complete_capital': int(sentence[index].upper()==sentence[index]),
        'prev_word':'' if index==0 else sentence[index-1],
        'next_word':'' if index==len(sentence)-1 else sentence[index+1],
        'is_numeric':int(sentence[index].isdigit()),
        'is_alphanumeric': int(bool((re.match('^(?=.*[0-9]$)(?=.*[a-zA-Z])',sentence[index])))),
        'prefix_1':sentence[index][0],
        'prefix_2': sentence[index][:2],
        'prefix_3':sentence[index][:3],
        'prefix_4':sentence[index][:4],
        'suffix_1':sentence[index][-1],
        'suffix_2':sentence[index][-2:],
        'suffix_3':sentence[index][-3:],
        'suffix_4':sentence[index][-4:],
        'lem':lemmatizer.lemmatize(sentence[index]),
        'has_apostrophe':int(bool(re.match('^(\w+\'\w+)',sentence[index]))),
        'word_has_hyphen': 1 if '-' in sentence[index] else 0}

In [63]:
# Functions fro getting the data ready in x, y data format has to parse the pair of (x,y) from a sentence that contains many of them
def get_word(pair):
    return [word for word, label in pair]
def make_data(sentence):
    xdata = []
    ydata = []
    
    for pair in sentence:
        parsed_sentence = get_word(pair)
        
        xdata.append([get_features(parsed_sentence, index) for index in range(len(pair))])
        ydata.append([label for word, label in pair])
    return xdata, ydata
        

In [64]:


xtrain , ytrain = make_data(train)
xtest , ytest = make_data(test)


In [65]:
xtrain[0]

[{'is_first_capital': 0,
  'is_first_word': 1,
  'is_last_word': 0,
  'is_complete_capital': 1,
  'prev_word': '',
  'next_word': 'It',
  'is_numeric': 0,
  'is_alphanumeric': 0,
  'prefix_1': '`',
  'prefix_2': '``',
  'prefix_3': '``',
  'prefix_4': '``',
  'suffix_1': '`',
  'suffix_2': '``',
  'suffix_3': '``',
  'suffix_4': '``',
  'lem': '``',
  'has_apostrophe': 0,
  'word_has_hyphen': 0},
 {'is_first_capital': 1,
  'is_first_word': 0,
  'is_last_word': 0,
  'is_complete_capital': 0,
  'prev_word': '``',
  'next_word': 'could',
  'is_numeric': 0,
  'is_alphanumeric': 0,
  'prefix_1': 'I',
  'prefix_2': 'It',
  'prefix_3': 'It',
  'prefix_4': 'It',
  'suffix_1': 't',
  'suffix_2': 'It',
  'suffix_3': 'It',
  'suffix_4': 'It',
  'lem': 'It',
  'has_apostrophe': 0,
  'word_has_hyphen': 0},
 {'is_first_capital': 0,
  'is_first_word': 0,
  'is_last_word': 0,
  'is_complete_capital': 0,
  'prev_word': 'It',
  'next_word': 'operate',
  'is_numeric': 0,
  'is_alphanumeric': 0,
  'prefix

In [66]:
labels = ['.', 'PRON', 'VERB', 'X', 'PRT', 'NOUN', 'DET', 'ADJ', 'ADP', 'ADV', 'NUM', 'CONJ']


In [67]:
# already deteremined that this is a good model fro this problem
# trying to determine the best hyperparameters
crf = CRF(
    
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.75),
    'c2': scipy.stats.expon(scale=0.2),
    'algorithm': ['lbfgs','l2sgd','arow']
}

# using accuracy for figuring out best parameters
acc_scorer = make_scorer(metrics.flat_accuracy_score)
#,
                       # average='weighted', labels=labels)

# below searchs for best parameters based on the parameter space
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=100,
                        scoring=acc_scorer)
rs.fit(xtrain, ytrain)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   22.4s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  4.0min finished


RandomizedSearchCV(cv=3,
                   estimator=CRF(all_possible_transitions=True,
                                 keep_tempfiles=None, max_iterations=100),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'algorithm': ['lbfgs', 'l2sgd', 'arow'],
                                        'c1': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001C07FB54D60>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001C07FBB6250>},
                   scoring=make_scorer(flat_accuracy_score), verbose=1)

In [68]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)

best params: {'algorithm': 'lbfgs', 'c1': 0.015376338904414984, 'c2': 0.1658155558740988}
best CV score: 0.9710130526001003


In [69]:
# using the best parameters found from above and fitting the model
# Then taking the and predicting based on the training data and getting accuracy for the training data
crfmodel = CRF(c1 =0.015674981907914637, c2 = 0.06028826022175581)#0.0192736642760659, c2 = 0.1641957022096317

crfmodel.fit(xtrain, ytrain)
ypred=crfmodel.predict(xtrain)
metrics.flat_accuracy_score(ytrain,ypred)

0.9975691400334742

In [70]:
#running the test data through the model and return the accuracy
y_pred=crfmodel.predict(xtest)
metrics.flat_accuracy_score(ytest,y_pred)

0.9758998188548476

In [71]:
#prints a couple metrics based on each label 
print(metrics.flat_classification_report(
    ytest, y_pred, labels=crf.classes_, digits=3
))

              precision    recall  f1-score   support

           .      1.000     0.999     0.999      2924
         ADJ      0.918     0.888     0.903      1627
         ADP      0.980     0.986     0.983      2484
         ADV      0.926     0.925     0.925       771
        CONJ      0.997     0.993     0.995       584
         DET      0.994     0.992     0.993      2211
        NOUN      0.969     0.979     0.974      7345
         NUM      0.997     0.989     0.993       912
        PRON      0.993     0.998     0.995       664
         PRT      0.981     0.974     0.978       809
        VERB      0.968     0.966     0.967      3383
           X      0.999     0.996     0.998      1680

    accuracy                          0.976     25394
   macro avg      0.977     0.974     0.975     25394
weighted avg      0.976     0.976     0.976     25394



In [72]:
#defining a new function for taking in a sentence and parsing it to get the data to be used for prediction
def get_testingfeatures(sentence, index):
    sentence = sentence.split(' ')
    return {
        'is_first_capital':int(sentence[index][0].isupper()),
        'is_first_word': int(index==0),
        'is_last_word':int(index==len(sentence)-1),
        'is_complete_capital': int(sentence[index].upper()==sentence[index]),
        'prev_word':'' if index==0 else sentence[index-1],
        'next_word':'' if index==len(sentence)-1 else sentence[index+1],
        'is_numeric':int(sentence[index].isdigit()),
        'is_alphanumeric': int(bool((re.match('^(?=.*[0-9]$)(?=.*[a-zA-Z])',sentence[index])))),
        'prefix_1':sentence[index][0],
        'prefix_2': sentence[index][:2],
        'prefix_3':sentence[index][:3],
        'prefix_4':sentence[index][:4],
        'suffix_1':sentence[index][-1],
        'suffix_2':sentence[index][-2:],
        'suffix_3':sentence[index][-3:],
        'suffix_4':sentence[index][-4:],
        'lem':lemmatizer.lemmatize(sentence[index]),
        'has_apostrophe':int(bool(re.match('^(\w+\'\w+)',sentence[index]))),
        'word_has_hyphen': 1 if '-' in sentence[index] else 0}

In [79]:
# takes a sentence and then iterates over it and gets all the words and puncutaion and runs it through the above function to get 
# the appropirate format for the model. 
# the sentence should be spaced between everything. so spaces inbetween words and punctuation
new_sentence = "My dog ran in the park ."
testingdata = []
testingdata.append([get_testingfeatures(new_sentence, index) for index in range(len(new_sentence.split(' ')))])
#testingdata

In [80]:
# runs the new data through the model and returns the results
crfmodel.predict(testingdata)

[['PRON', 'NOUN', 'VERB', 'ADP', 'DET', 'NOUN', '.']]

In [75]:
from collections import Counter
Counter(crfmodel.transition_features_).most_common(20)

[(('ADJ', 'NOUN'), 4.460021),
 (('VERB', 'PRT'), 3.285621),
 (('NOUN', 'NOUN'), 3.044629),
 (('NOUN', 'PRT'), 2.886757),
 (('NOUN', 'VERB'), 2.834285),
 (('ADP', 'NOUN'), 2.685314),
 (('DET', 'NOUN'), 2.291516),
 (('NUM', 'NOUN'), 2.255194),
 (('X', 'VERB'), 2.246067),
 (('PRON', 'VERB'), 2.087049),
 (('NOUN', 'ADP'), 1.966258),
 (('ADV', 'VERB'), 1.953971),
 (('ADV', 'ADJ'), 1.92981),
 (('VERB', 'NOUN'), 1.871431),
 (('ADP', 'PRON'), 1.758661),
 (('NOUN', 'CONJ'), 1.67907),
 (('NOUN', '.'), 1.670311),
 (('ADJ', 'ADJ'), 1.48703),
 (('ADV', 'ADV'), 1.45919),
 (('CONJ', 'NOUN'), 1.407435)]

In [76]:
Counter(crfmodel.transition_features_).most_common()[-20:]

[(('X', 'DET'), -1.249699),
 (('X', 'NUM'), -1.307248),
 (('CONJ', '.'), -1.345672),
 (('X', 'NOUN'), -1.352653),
 (('DET', 'DET'), -1.353117),
 (('ADJ', 'PRON'), -1.394371),
 (('PRT', '.'), -1.422574),
 (('X', 'ADJ'), -1.42733),
 (('ADP', 'PRT'), -1.438229),
 (('ADV', 'X'), -1.450862),
 (('PRT', 'PRT'), -1.503432),
 (('PRON', 'DET'), -1.742286),
 (('PRON', 'PRT'), -1.767708),
 (('DET', 'ADP'), -1.894233),
 (('X', 'PRT'), -1.985065),
 (('PRT', 'NUM'), -2.065554),
 (('.', 'PRT'), -2.972698),
 (('ADP', 'X'), -3.551786),
 (('CONJ', 'X'), -3.808095),
 (('DET', 'PRT'), -4.878475)]